# **Loading Necessary Libraries**

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell',set_css)

In [ ]:
!pip install transformers
!pip install newspaper3k
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 7.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 75.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.wh

In [ ]:
from transformers import pipeline
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from heapq import nlargest
nltk.download('punkt')
nltk.download('stopwords')
from newspaper import Article

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


# **Define Extractive Text Summarization Function**

In [ ]:
def summarize(text, num_sentences=10):
    sentences = sent_tokenize(text)
    words = word_tokenize(text.lower())

    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalnum() and word not in stop_words]

    freq_dist = FreqDist(words)

    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        sentence_words = word_tokenize(sentence.lower())
        sentence_score = 0
        for word in sentence_words:
            if word in freq_dist:
                sentence_score += freq_dist[word]
        sentence_scores[i] = sentence_score

    summary_indexes = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    summary_indexes.sort()
    summary = [sentences[i] for i in summary_indexes]
    return '.'.join(summary)

# **Fetch News Article**

In [ ]:
url = 'https://www.vox.com/the-goods/2023/2/7/23588382/gautam-adani-hindenburg-research-india-modi-short-seller'
article = Article(url)
article.download()
article.parse()
article.nlp()

# **Generate Extractive Summary**

In [ ]:
summ = summarize(article.text)
print(summ)

On January 24, New York-based Hindenburg Research published a scathing 100-plus-page report making the case that Adani is “pulling the largest con in corporate history.” In it, Hindenburg, which was founded by activist short seller Nathan Anderson in 2017, alleges Adani Group has engaged in “brazen accounting fraud, stock manipulation, and money laundering” over the course of decades “with the help of enablers in government and a cottage industry of international companies that facilitate these activities.” It takes aim at seven companies listed by Adani that have seen their stock prices surge over the past three years..Shares of Adani’s flagship firm, Adani Enterprises, have plunged, as have the stock prices of other Adani companies..Much of Adani’s wealth is tied up in his companies, and as their prices have fallen, so has his net worth

Much of Adani’s wealth is tied up in his companies, and as their prices have fallen, so has his net worth..Adani has become one of the “poster child

In [ ]:
# Rogue Score for Extractive Summary
from rouge_score import rouge_scorer
li = []
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores2 = scorer.score(article.text, summ)
print("Rouge1 -> ", scores2['rouge1'])
print("Rouge2 -> ", scores2['rouge2'])
print("RougeL -> ", scores2['rougeL'])
li.append(scores2['rouge1'])
print(li)

Rouge1 ->  Score(precision=1.0, recall=0.2205005959475566, fmeasure=0.361328125)
Rouge2 ->  Score(precision=0.983754512635379, recall=0.21661367249602545, fmeasure=0.35504885993485347)
RougeL ->  Score(precision=1.0, recall=0.2205005959475566, fmeasure=0.361328125)
[Score(precision=1.0, recall=0.2205005959475566, fmeasure=0.361328125)]


# **Generate Abstractive Summary**

In [ ]:
summary_text = summarizer(summ, max_length=400, min_length=100, do_sample=False)[0]['summary_text']
print("\n\n\n")
print(summary_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (776 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.9/dist-packages/transformers/generation/tf_utils.py:745: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(






a new report alleges that Adani is "pulling the largest con in corporate history" it takes aim at seven companies listed by Adani that have seen their stock prices surge . adani has become one of the "poster children" for india's economic growth prowess in recent years . an implosion of a conglomerate like Adani's would "dramatically raise" the country's risk profile, experts say .


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores2 = scorer.score(article.text, summary_text)
print("Rouge1 -> ", scores2['rouge1'])
print("Rouge2 -> ", scores2['rouge2'])
print("RougeL -> ", scores2['rougeL'])
scores2['rougeL'][1]

Rouge1 ->  Score(precision=0.9846153846153847, recall=0.02542709574890743, fmeasure=0.049573973663826494)
Rouge2 ->  Score(precision=0.890625, recall=0.022655007949125595, fmeasure=0.044186046511627906)
RougeL ->  Score(precision=0.9692307692307692, recall=0.025029797377830752, fmeasure=0.048799380325329204)


0.025029797377830752

In [ ]:
len(summary_text)

384

In [ ]:
def getArticle(text):
  url = text
  article = Article(url)
  article.download()
  article.parse()
  article.nlp()
  return article.text

def generateSummary(url):
  articleText = getArticle(url)
  summ = summarize(articleText)
  summary_text = summarizer(summ, max_length=400, min_length=100, do_sample=False)[0]['summary_text']
  return summary_text

In [ ]:
summary2 = generateSummary("https://timesofindia.indiatimes.com/india/sharp-covid-surge-this-week-daily-cases-top-3k-now-doubling-in-7-days/articleshow/99128392.cms")
print(summary2)

india logged 3,016 fresh cases on Wednesday, the highest single-day tally since October 2 last year . the last two days have seen steep spikes in cases in Kerala, Delhi, Himachal and Haryana . daily positivity rate has surged to 2.73% from less than 1% about a month ago . 'breakthrough infections' are usually mild or asymptomatic, but in some cases, they can still cause severe illness and death .


### **Testing Rouge Scores**

In [ ]:
li = ['https://www.vox.com/politics/2023/4/4/23668602/trump-arrested-arraignment-indictment-court-new-york',
      'https://timesofindia.indiatimes.com/india/rajya-sabha-chairman-allowed-subversion-of-standing-committees-basic-function-jairam-ramesh/articleshow/99288581.cms',
      'https://timesofindia.indiatimes.com/business/india-business/rbi-mpc-meet-repo-rate-loan-emis-shaktikanta-das/articleshow/99284097.cms',
      'https://timesofindia.indiatimes.com/business/india-business/rbi-mpc-meet-repo-rate-loan-emis-shaktikanta-das/articleshow/99284097.cms',
      'https://timesofindia.indiatimes.com/business/international-business/germanys-lufthansa-to-sell-catering-business-lsg-group/articleshow/99269713.cms',
      'https://timesofindia.indiatimes.com/business/international-business/britains-hilton-food-names-former-co-op-boss-murrells-as-ceo/articleshow/99263656.cms',
      'https://www.vox.com/politics/2023/4/3/23668444/mcdonalds-layoffs-corporate-economy',
      'https://www.vox.com/politics/2023/4/1/23664751/trump-indictment-alvin-bragg-stormy-daniels',
      'https://www.vox.com/23670278/forbes-billionaire-ranking-2023-crypto-down-lebron-james-jimmy-buffett-donald-trump',
      'https://www.vox.com/politics/2023/4/1/23664751/trump-indictment-alvin-bragg-stormy-daniels']

In [ ]:
for x in li:
  articleText = getArticle(x)
  extSum = summarize(articleText)
  absSum = summarizer(extSum, max_length=400, min_length=100, do_sample=False)[0]['summary_text']
  print("Extractive -> ")
  scores2 = scorer.score(articleText, extSum)
  print("Rouge1 -> ", scores2['rouge1'])
  print("Rouge2 -> ", scores2['rouge2'])
  print("RougeL -> ", scores2['rougeL'])

  print("Abstraactive -> ")
  scores2 = scorer.score(articleText, absSum)
  print("Rouge1 -> ", scores2['rouge1'])
  print("Rouge2 -> ", scores2['rouge2'])
  print("RougeL -> ", scores2['rougeL'])

Extractive -> 
Rouge1 ->  Score(precision=1.0, recall=0.22845691382765532, fmeasure=0.3719412724306688)
Rouge2 ->  Score(precision=0.9846153846153847, recall=0.22456140350877193, fmeasure=0.3657142857142857)
RougeL ->  Score(precision=1.0, recall=0.22845691382765532, fmeasure=0.3719412724306688)
Abstraactive -> 
Rouge1 ->  Score(precision=1.0, recall=0.04008016032064128, fmeasure=0.07707129094412331)
Rouge2 ->  Score(precision=0.9367088607594937, recall=0.037092731829573934, fmeasure=0.07135969141755062)
RougeL ->  Score(precision=0.975, recall=0.03907815631262525, fmeasure=0.07514450867052022)
Extractive -> 
Rouge1 ->  Score(precision=1.0, recall=1.0, fmeasure=1.0)
Rouge2 ->  Score(precision=1.0, recall=1.0, fmeasure=1.0)
RougeL ->  Score(precision=1.0, recall=1.0, fmeasure=1.0)
Abstraactive -> 
Rouge1 ->  Score(precision=0.975, recall=0.11504424778761062, fmeasure=0.20580474934036938)
Rouge2 ->  Score(precision=0.6835443037974683, recall=0.07976366322008863, fmeasure=0.14285714285714

Your max_length is set to 400, but you input_length is only 249. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=124)


Extractive -> 
Rouge1 ->  Score(precision=1.0, recall=1.0, fmeasure=1.0)
Rouge2 ->  Score(precision=1.0, recall=1.0, fmeasure=1.0)
RougeL ->  Score(precision=1.0, recall=1.0, fmeasure=1.0)
Abstraactive -> 
Rouge1 ->  Score(precision=0.9615384615384616, recall=0.423728813559322, fmeasure=0.5882352941176471)
Rouge2 ->  Score(precision=0.8571428571428571, recall=0.375, fmeasure=0.5217391304347825)
RougeL ->  Score(precision=0.8333333333333334, recall=0.3672316384180791, fmeasure=0.5098039215686275)


Your max_length is set to 400, but you input_length is only 237. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=118)


Extractive -> 
Rouge1 ->  Score(precision=1.0, recall=1.0, fmeasure=1.0)
Rouge2 ->  Score(precision=1.0, recall=1.0, fmeasure=1.0)
RougeL ->  Score(precision=1.0, recall=1.0, fmeasure=1.0)
Abstraactive -> 
Rouge1 ->  Score(precision=0.9861111111111112, recall=0.42771084337349397, fmeasure=0.5966386554621849)
Rouge2 ->  Score(precision=0.8591549295774648, recall=0.3696969696969697, fmeasure=0.5169491525423728)
RougeL ->  Score(precision=0.8333333333333334, recall=0.3614457831325301, fmeasure=0.504201680672269)
Extractive -> 
Rouge1 ->  Score(precision=1.0, recall=0.4762532981530343, fmeasure=0.645218945487042)
Rouge2 ->  Score(precision=0.9861111111111112, recall=0.46895640686922063, fmeasure=0.6356311548791406)
RougeL ->  Score(precision=1.0, recall=0.4762532981530343, fmeasure=0.645218945487042)
Abstraactive -> 
Rouge1 ->  Score(precision=1.0, recall=0.10686015831134564, fmeasure=0.1930870083432658)
Rouge2 ->  Score(precision=0.8875, recall=0.09379128137384413, fmeasure=0.169653524492